In [1]:
import config
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [3]:
train = pd.read_parquet(os.path.join(config.data_processed_dir , 'train.parquet'))
valid = pd.read_parquet(os.path.join(config.data_processed_dir , 'valid.parquet'))

In [65]:
articles = pd.read_parquet(os.path.join(config.data_raw_dir , 'articles.parquet'))


In [66]:
articles

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105537,953450001,953450,5pk regular Placement1,302,Socks,Socks & Tights,1010014,Placement print,9,Black,...,Socks Bin,F,Menswear,3,Menswear,26,Men Underwear,1021,Socks and Tights,Socks in a fine-knit cotton blend with a small...
105538,953763001,953763,SPORT Malaga tank,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,2,H&M+,1005,Jersey Fancy,Loose-fitting sports vest top in ribbed fast-d...
105539,956217002,956217,Cartwheel dress,265,Dress,Garment Full body,1010016,Solid,9,Black,...,Jersey,A,Ladieswear,1,Ladieswear,18,Womens Trend,1005,Jersey Fancy,"Short, A-line dress in jersey with a round nec..."
105540,957375001,957375,CLAIRE HAIR CLAW,72,Hair clip,Accessories,1010016,Solid,9,Black,...,Small Accessories,D,Divided,2,Divided,52,Divided Accessories,1019,Accessories,Large plastic hair claw.


In [68]:
# select only string columns
# get names of s
articles = articles[['article_id', 'prod_name', 'detail_desc']]

# join prod_name and prod_desc
articles['text'] = articles['prod_name'] + ' ' + articles['detail_desc']
articles['text'] = articles['text'].str.lower()
articles = articles[['article_id', 'text']]

/tmp/ipykernel_14317/710849988.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['text'] = articles['prod_name'] + ' ' + articles['detail_desc']
/tmp/ipykernel_14317/710849988.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['text'] = articles['text'].str.lower()


In [7]:
# select only the columns we need
train = train[['customer_id', 'article_id']]
valid = valid[['customer_id', 'article_id']]
# concat train and valid
train = pd.concat([train, valid], axis=0)

In [8]:
# groupby customer_id and aggregate article_id into a list, then split the lists into groups of 2 consecutive articles
train = train.groupby('customer_id')['article_id'].agg(list).apply(lambda x: [x[i:i+2] for i in range(len(x)-1)]).explode().reset_index()
train = train[train['article_id'].notna()]
# explode article_id into 2 columns article_1 and article_2
train = train.join(pd.DataFrame(train.pop('article_id').tolist(), columns=['article_1', 'article_2']))
# drop na values again
train = train[train['article_1'].notna()]
train = train[train['article_2'].notna()]
# drop rows where article_1 and article_2 are the same
train = train[train['article_1'] != train['article_2']]
train['label'] = 1

In [15]:
train

,customer_id,article_1,article_2,label
2,1,599580055.0,811835004.0,1
6,1,811835004.0,723529001.0,1
7,1,723529001.0,559630026.0,1
8,1,559630026.0,599580083.0,1
9,1,599580083.0,811927004.0,1
...,...,...,...,...
11253154,1368904,884081001.0,794819001.0,1
11253155,1368904,794819001.0,762846027.0,1
11253156,1368904,866755002.0,840360003.0,1
11253157,1368904,840360003.0,866755002.0,1


In [16]:
# now we need to create negative samples, for each article_1 we need to randomly select N article_2 that is not the same as article_1
# N is the number of negative samples we want
N = 300

# create a copy of train
train_negative = train.copy()

# create a list of all article_id
article_ids = train['article_1'].unique()

# for each article_id, randomly select N article_id that is not the same as article_id
negative_samples = []
for article_id in article_ids:
    do_not_select = train[train['article_1'] == article_id]['article_2'].unique()
    # randomly select N article_id that is not in do_not_select
    negative_samples.extend(np.random.choice(np.setdiff1d(article_ids, do_not_select), N, replace=False))
# create a dataframe from the negative samples
train_negative = pd.DataFrame(negative_samples, columns=['article_2'])
train_negative['article_1'] = article_ids.repeat(N)
train_negative['label'] = 0

train_negative


,article_2,article_1,label
0,869005001.0,599580055.0,0
1,866111001.0,599580055.0,0
2,856667005.0,599580055.0,0
3,739461002.0,599580055.0,0
4,679011009.0,599580055.0,0
...,...,...,...
14996995,662369058.0,795358001.0,0
14996996,841808001.0,795358001.0,0
14996997,687635018.0,795358001.0,0
14996998,805000002.0,795358001.0,0


In [20]:
# merge train and train_negative    
train = pd.concat([train, train_negative], ignore_index=True)
train.drop_duplicates(inplace=True)

In [24]:
# split 30% of train as valid
valid = train.sample(frac=0.3, random_state=42)
train = train.drop(valid.index)

In [25]:
train.to_parquet(os.path.join(config.data_processed_dir , 'train_pairs.parquet'), index=False)
valid.to_parquet(os.path.join(config.data_processed_dir , 'valid_pairs.parquet'), index=False)

In [75]:
train = pd.read_parquet(os.path.join(config.data_processed_dir , 'train_pairs.parquet'))
valid = pd.read_parquet(os.path.join(config.data_processed_dir , 'valid_pairs.parquet'))

In [76]:
def merge_articles(df, articles, article_column, text_column):
    df = df.merge(articles, left_on=article_column, right_on='article_id', how='left')
    df = df.rename(columns={'text': text_column})
    return df.drop(columns=['article_id'])

# Use the helper function to merge train and valid with article_1 and article_2
train = merge_articles(train, articles, 'article_1', 'text_1')
train = merge_articles(train, articles, 'article_2', 'text_2')

valid = merge_articles(valid, articles, 'article_1', 'text_1')
valid = merge_articles(valid, articles, 'article_2', 'text_2')


In [85]:
train.dropna(inplace=True)
valid.dropna(inplace=True)

In [3]:
print(valid.label.mean())
print(train.label.mean())


0.35854529196213414
0.35856163810206754


In [88]:
df = pd.concat([train, valid], ignore_index=True)
item_ids = set(df['article_1'].unique()).union(set(df['article_2'].unique()))
# Create a dictionary that maps each item ID to a unique index
vocab = {item_id: i for i, item_id in enumerate(item_ids)}
num_items = len( set(df['article_1'].unique()).union(set(df['article_2'].unique())) )
del item_ids, df

In [100]:
from transformers import BertTokenizer

class PairDataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        # Create a dictionary that maps each item ID to a unique index
        self.vocab = vocab
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', truncation=True, padding=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        # get article_id1 and article_id2 and labels
        article_id1 = row['article_1']
        article_id2 = row['article_2']
        label = row['label']
        # convert to torch tensors
        article_id1 = torch.tensor(self.vocab[article_id1], dtype=torch.long)
        article_id2 = torch.tensor(self.vocab[article_id2], dtype=torch.long)
        label = torch.tensor(label, dtype=torch.float)

        tokens1 = self.tokenizer(row['text_1'], return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        tokens2 = self.tokenizer(row['text_2'], return_tensors='pt', max_length=128, padding='max_length', truncation=True)
        return article_id1, article_id2, tokens1['input_ids'].squeeze(), tokens1['attention_mask'].squeeze(), tokens2['input_ids'].squeeze(), tokens2['attention_mask'].squeeze(), label
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
dataset = PairDataset(train, vocab)
data_loader = DataLoader(dataset, batch_size=1024, shuffle=True)
valid_dataset = PairDataset(valid, vocab)
valid_data_loader = DataLoader(valid_dataset, batch_size=4000, shuffle=False)

Using device: cuda


In [103]:
import torch
from torch import nn, optim


class Item2Vec(nn.Module):
    def __init__(self, num_items, embedding_dim):
        super().__init__()
        # embedding layer
        self.embeddings = nn.Embedding(num_items, embedding_dim)

        transformer_layer = nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=4)
        self.transformer = nn.TransformerEncoder(transformer_layer, num_layers=4)

        # dense layer
        self.linear = nn.Linear(embedding_dim * 2, 512)
        # activation function
        self.act = nn.ReLU()
        # dropout
        self.dropout = nn.Dropout(0.2)
        # output layer
        self.output = nn.Linear(512, embedding_dim)
        # output activation
        self.output_act = nn.Sigmoid()

    def forward(self, item1, input_ids, attention_mask):
        # pass through transforme
        text1 = self.transformer(input_ids, attention_mask)

        embed1 = self.embeddings(item1)

        embed1 = torch.cat((embed1, text1), dim=1)

        embed1 = self.dropout(embed1)
        # pass through dense layer
        dense1 = self.linear(embed1)
        # pass through activation function
        act1 = self.act(dense1)
        # pass through dropout
        # pass through output layer
        output = self.output(act1)
        # pass through output activation
        output = self.output_act(output)

        return output

def loss_function(output, target):
    return nn.BCEWithLogitsLoss()(output, target)

def train_model(model, data_loader, optimizer, num_epochs):
    print(device)
    model = model.to(device)
    
    for epoch in range(num_epochs):
        # switch model to training mode
        model.train()
        with tqdm(total=len(data_loader)) as progress_bar:
            for i, (item1, item2, input_ids1, attention_mask1, input_ids2, attention_mask2, label) in enumerate(data_loader):
                optimizer.zero_grad()
                output1 = model(item1.to(device), input_ids1.float().to(device), attention_mask1.float().to(device))
                output2 = model(item2.to(device), input_ids2.to(device), attention_mask2.to(device))

                output1 = output1.view(output1.shape[0], 1, output1.shape[1])  # shape: (batch_size, 1, embedding_dim)
                output2 = output2.view(output2.shape[0], output2.shape[1], 1)  # shape: (batch_size, embedding_dim, 1)
                # Compute the dot product
                output = torch.bmm(output1, output2)  # shape: (batch_size, 1, 1)

                # Reshape the output to be two-dimensional
                output = output.view(output.shape[0], -1)  # shape: (batch_size, 1)
                target = target.view(target.shape[0], -1)  # shape: (batch_size, 1)

                loss = loss_function(output, target.float().to(device))

                loss.backward()
                optimizer.step()
                if i % 10 == 0:
                    progress_bar.set_postfix(loss=loss.item())
                    progress_bar.update(10)

               
            
        # compute total loss and accuracy
        total_loss = 0
        total_accuracy = 0

        # switch model to evaluation mode
        model.eval()

        with torch.no_grad():
            with tqdm(total=len(valid_data_loader)) as progress_bar:
                for item1, item2, target in valid_data_loader:
                    output1 = model(item1.to(device))
                    output2 = model(item2.to(device))

                    output1 = output1.view(output1.shape[0], 1, output1.shape[1])  # shape: (batch_size, 1, embedding_dim)
                    output2 = output2.view(output2.shape[0], output2.shape[1], 1)  # shape: (batch_size, embedding_dim, 1)
                    # Compute the dot product
                    output = torch.bmm(output1, output2)  # shape: (batch_size, 1, 1)

                    # Reshape the output to be two-dimensional
                    output = output.view(output.shape[0], -1)  # shape: (batch_size, 1)
                    target = target.view(target.shape[0], -1)  # shape: (batch_size, 1)

                    loss = loss_function(output, target.float().to(device))
                    
                    # compute accuracy
                    output = output.detach().cpu().numpy()
                    target = target.detach().cpu().numpy()
                    accuracy = ((output > 0) == target).mean()
                    total_loss += loss.item()
                    total_accuracy += accuracy
                    progress_bar.update(1)

        print('Epoch: {}, Loss: {:.4f}, Accuracy: {:.4f}'.format(epoch+1, total_loss/len(valid_data_loader), total_accuracy/len(valid_data_loader)))




model = Item2Vec(num_items=num_items, embedding_dim=128)
train_model(model, data_loader, optimizer=torch.optim.Adam(model.parameters(), lr=0.001), num_epochs=10)

/home/joao/Desktop/item2vec/.venv-dev/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


cuda


  0%|          | 0/5724 [00:00<?, ?it/s]


RuntimeError: The shape of the 2D attn_mask is torch.Size([1024, 128]), but should be (1024, 1024).